In [1]:
import json

import numpy as np
import torch
import matplotlib.pyplot as plt

# Increase Experts 

In [2]:
# === OvA ===
n_classes = 10
confs = []
exps = []
true = []
path = "ova_increase_experts/"
n_experts = [1, 2, 4, 6, 8]
for n in n_experts:
    model_name = '_' + str(n) + '_experts'
    with open(path + 'confidence_multiple_experts' + model_name + '.txt', 'r') as f:
        conf = json.loads(json.load(f))
    with open(path + 'expert_predictions_multiple_experts' + model_name + '.txt', 'r') as f:
        exp_pred = json.loads(json.load(f))
    with open(path + 'true_label_multiple_experts' + model_name + '.txt', 'r') as f:
        true_label = json.loads(json.load(f))
    true.append(true_label['test'])
    exps.append(exp_pred['test'])
    c = torch.tensor(conf['test'])
    print(c.shape)
    # DANI Correction ===
    c = c.sigmoid()
    # DANI Correction ===
    confs.append(c)

torch.Size([9216, 11])
torch.Size([9216, 12])
torch.Size([9216, 14])
torch.Size([9216, 16])
torch.Size([9216, 18])


# Increase confidence 

In [5]:
# === OvA ===
n_classes = 10
random_expert_idx = 0
confs = []
exps = []
true = []
path = "ova_increase_confidence/"
n_experts = 4
p_experts = [0.2, 0.4, 0.6, 0.8, 0.95]
for p in p_experts:
    model_name = '_' + str(p) + '_confidence'
    with open(path + 'confidence_multiple_experts' + model_name + '.txt', 'r') as f:
        conf = json.loads(json.load(f))
    with open(path + 'expert_predictions_multiple_experts' + model_name + '.txt', 'r') as f:
        exp_pred = json.loads(json.load(f))
    with open(path + 'true_label_multiple_experts' + model_name + '.txt', 'r') as f:
        true_label = json.loads(json.load(f))
    true.append(true_label['test'])
    exps.append(exp_pred['test'])
    c = torch.tensor(conf['test'])
    # DANI Correction ===
    c = c.sigmoid()
    # DANI Correction ===
    confs.append(c)

In [6]:
# 1 expert rando, 3 with prob 0.8 correct
probs = confs[-2]
experts = exps[-2]
y_true = true[-2]

In [7]:
n_val = int(0.8 * len(y_true))
n_test = len(y_true) - n_val
print("N val:{}".format(n_val))
print("N test:{}".format(n_test))

N val:7372
N test:1844


# Validation

In [88]:
## 2. get Q_hat
probs_val = probs[:n_val, 10:]
experts_val = experts[::-1]  # IMPORTANT! swap to match prob ordering
experts_val = [exp[:n_val] for exp in experts_val]
y_true_val = y_true[:n_val]

_, predicted = torch.max(probs[:n_val].data, 1)
r = (predicted >= n_classes - n_experts)

# === Only on deferred samples
probs_val = probs_val[r]
experts_val = [np.array(exp)[r] for exp in experts_val]
y_true_val = np.array(y_true_val)[r]

# Model expert probs ===
# Sort J model outputs for experts
probs_experts = probs[:n_val, 10:]
probs_experts = probs_experts[r]
sort, pi = probs_experts.sort(dim=1, descending=True)

# Correctness experts ===
# Check if experts are correct 
correct_exp = (np.array(experts_val) == np.array(y_true_val)).T
# idx for correct experts: [[0,1,2], [1,2], [], ...]
correct_exp_idx = [np.where(correct_exp_i)[0] for correct_exp_i in correct_exp]

# obtain the last expert to be retrieved. If empty, then add all values.
# indexes are not the real expert index, but the sorted indexes, e.g. [[1, 0 ,2],  [1,0], [], ...]
pi_corr_exp = [probs_experts[i, corr_exp].sort(descending=True)[1] for i, corr_exp in enumerate(correct_exp)]
pi_corr_exp_stop = [pi_corr_exp_i[-1] if len(pi_corr_exp_i)!=0 else -1 for pi_corr_exp_i in pi_corr_exp]  # last expert

# obtain real expert index back, e.g. [2,1,-1,...]
pi_stop = [correct_exp_idx[i][pi_corr_exp_stop_i] if len(correct_exp_idx[i])!=0 else -1 for i, pi_corr_exp_stop_i in enumerate(pi_corr_exp_stop)]


# =========
n_val = n_val
alpha = 0.1
scores = sort.cumsum(dim=1).gather(1, pi.argsort(1))[range(len(torch.tensor(pi_stop))), torch.tensor(pi_stop)]
qhat = torch.quantile(scores, np.ceil((n_val + 1) * (1 - alpha)) / n_val, interpolation="higher")

qhat

tensor(2.3296)

In [86]:
torch.quantile(scores, np.ceil((n_val + 1) * (1 - alpha)) / n_val)


tensor(0.3074)

# Calculate Q hat

In [12]:
np.array(experts_val)[:,:10]

array([[6, 0, 8, 7, 4, 5, 3, 2, 4, 2],
       [5, 3, 5, 7, 6, 1, 9, 7, 2, 2],
       [8, 7, 6, 8, 7, 1, 0, 8, 6, 4],
       [9, 8, 6, 2, 6, 1, 3, 9, 5, 3]])

In [13]:
y_true_val[:10]

array([8, 8, 6, 6, 6, 1, 9, 7, 9, 8])

In [14]:
correct_exp = (np.array(experts_val) == np.array(y_true_val)).T
correct_exp_idx = [np.where(correct_exp_i)[0] for correct_exp_i in correct_exp]
correct_exp_idx[:10]

[array([2]),
 array([3]),
 array([2, 3]),
 array([], dtype=int64),
 array([1, 3]),
 array([1, 2, 3]),
 array([1]),
 array([1]),
 array([], dtype=int64),
 array([], dtype=int64)]

In [216]:
correct_exp[0]

array([False, False,  True, False])

In [97]:
correct_exp

array([[ True, False, False, False],
       [ True,  True,  True, False],
       [ True,  True,  True, False],
       ...,
       [ True,  True, False, False],
       [ True, False, False, False],
       [False,  True, False, False]])

In [94]:
correct_exp_idx

[array([0]),
 array([0, 1, 2]),
 array([0, 1, 2]),
 array([0, 1, 2]),
 array([1, 2]),
 array([], dtype=int64),
 array([3]),
 array([], dtype=int64),
 array([], dtype=int64),
 array([0, 1, 2]),
 array([0, 1, 2]),
 array([], dtype=int64),
 array([0, 1, 2]),
 array([], dtype=int64),
 array([0]),
 array([1]),
 array([1]),
 array([1, 2]),
 array([0]),
 array([0, 2]),
 array([1]),
 array([0, 1, 2]),
 array([], dtype=int64),
 array([0, 2]),
 array([0, 2]),
 array([0]),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([0, 2]),
 array([], dtype=int64),
 array([1]),
 array([2, 3]),
 array([2]),
 array([], dtype=int64),
 array([0, 2]),
 array([], dtype=int64),
 array([0, 1, 2]),
 array([], dtype=int64),
 array([3]),
 array([0, 3]),
 array([0, 1, 2]),
 array([0, 1]),
 array([0, 1, 2]),
 array([], dtype=int64),
 array([2]),
 array([], dtype=int64),
 array([], dtype=int64),
 array([1]),
 array([0, 1, 2]),
 array([0]),
 array([], dtype=int64),

In [16]:
len(experts)

4

# Test

In [445]:
n_classes = 10
n_experts = 4

In [153]:
probs_test = probs[n_val:, 10:]
experts_test = [exp[n_val:] for exp in experts]
y_true_test = y_true[n_val:]

# 2.b Sort J model outputs for experts
probs_experts = probs[n_val:, 10:]
sort_test, pi_test = probs_experts.sort(dim=1, descending=True)

# get first index out of the quantile region
pi_stop = (sort_test.cumsum(dim=1) >= qhat).int().argmax(dim=1)
prediction_sets = [pi_test[i][:(sizes[i])] for i in range(sizes.shape[0])]  # not allow empty sets


In [191]:
y_true_test[237]

0

In [190]:
probs[n_val:][237]

tensor([9.9252e-01, 7.3340e-04, 9.6698e-04, 5.8992e-04, 4.8714e-04, 3.1410e-04,
        1.6704e-03, 4.7459e-04, 1.2163e-02, 4.1350e-04, 7.6708e-01, 7.5882e-01,
        7.6625e-01, 1.2735e-01])

In [155]:
(sort_test.cumsum(dim=1) >= qhat).int().argmax(dim=1)[:10]

tensor([2, 2, 2, 0, 2, 2, 2, 0, 0, 2])

In [186]:
prediction_sets[237]

tensor([0, 2, 1])

In [187]:
correct_exp[237]

array([ True,  True, False,  True])

In [123]:
(sort_test.cumsum(dim=1) <= qhat)

tensor([[ True,  True, False, False],
        [ True,  True, False, False],
        [ True,  True, False, False],
        ...,
        [ True,  True, False, False],
        [ True,  True,  True,  True],
        [ True,  True, False, False]])

In [115]:
sort_test.cumsum(dim=1)

tensor([[0.8437, 1.6598, 2.4276, 2.5336],
        [0.8149, 1.6185, 2.4164, 2.5470],
        [0.8606, 1.6933, 2.4441, 2.5251],
        ...,
        [0.8567, 1.6991, 2.5215, 2.6117],
        [0.2486, 0.4933, 0.7304, 0.8268],
        [0.8283, 1.6240, 2.4011, 2.5174]])

In [108]:
correct_exp_idx[:10]

[array([0, 1, 2]),
 array([1, 2]),
 array([0, 1, 2, 3]),
 array([0, 1, 2]),
 array([0, 1, 2]),
 array([0, 1, 2]),
 array([0, 1, 2]),
 array([0]),
 array([], dtype=int64),
 array([0, 1])]

In [109]:
prediction_sets[:10]

[tensor([1, 0, 2]),
 tensor([2, 0, 1]),
 tensor([1, 0, 2]),
 tensor([1]),
 tensor([0, 2, 1]),
 tensor([1, 0, 2]),
 tensor([1, 0, 2]),
 tensor([1]),
 tensor([1]),
 tensor([0, 2, 1])]

In [23]:
_, predicted = torch.max(probs[n_val:].data, 1)
r = (predicted >= n_classes - n_experts)

In [24]:
r = (predicted >= n_classes - n_experts)

In [98]:
np.array(experts_test)

array([[5, 4, 3, ..., 2, 6, 5],
       [5, 0, 3, ..., 4, 9, 5],
       [5, 0, 3, ..., 8, 9, 5],
       [6, 7, 3, ..., 1, 3, 7]])

In [93]:

# Check if experts are correct 
experts_test = experts_test[::-1]
correct_exp = (np.array(experts_test) == np.array(y_true_test)).T
# Swap order to match confidence ordering

# idx for correct experts: [[0,1,2], [1,2], [], ...]
correct_exp_idx = [np.where(correct_exp_i)[0] for correct_exp_i in correct_exp]


In [94]:
correct_exp_idx

[array([0, 1, 2]),
 array([1, 2]),
 array([0, 1, 2, 3]),
 array([0, 1, 2]),
 array([0, 1, 2]),
 array([0, 1, 2]),
 array([0, 1, 2]),
 array([0]),
 array([], dtype=int64),
 array([0, 1]),
 array([2]),
 array([0, 1, 2]),
 array([0, 1]),
 array([], dtype=int64),
 array([0, 1]),
 array([0, 1, 2]),
 array([], dtype=int64),
 array([], dtype=int64),
 array([1]),
 array([0, 1, 2]),
 array([0]),
 array([1, 2]),
 array([1, 2]),
 array([1, 2]),
 array([0, 1, 2]),
 array([0, 1, 2]),
 array([0, 1, 2]),
 array([0]),
 array([0, 1, 2, 3]),
 array([0, 1, 2]),
 array([0, 1, 2]),
 array([0, 2]),
 array([], dtype=int64),
 array([0, 1, 2, 3]),
 array([2]),
 array([], dtype=int64),
 array([0, 1, 2]),
 array([], dtype=int64),
 array([0, 1, 2, 3]),
 array([0, 1, 2]),
 array([0]),
 array([1, 2]),
 array([], dtype=int64),
 array([0, 1, 2]),
 array([0, 1, 2, 3]),
 array([1, 2]),
 array([1]),
 array([0, 1, 2]),
 array([1, 2]),
 array([0, 1, 2]),
 array([0, 1, 2]),
 array([], dtype=int64),
 array([0, 2]),
 array([